In [1]:
import numpy as np
import pandas as pd
import os
import h5py
from exabiome.nn.loader import read_dataset, LazySeqDataset
import argparse
import torch

In [4]:
path = '/global/homes/a/azaidi'
#os.listdir(path)

In [5]:
f_path = path +'/ar122_r202.toy.input.h5'
f_path

'/global/homes/a/azaidi/ar122_r202.toy.input.h5'

In [6]:
#dataset, io = read_dataset(f_path)

In [7]:
hparams = argparse.Namespace(**{'load': False,
                            'window': 4096,
                            'step': 4096,
                             'classify': True,
                               'tgt_tax_lvl': "phylum",
                               'fwd_only': True})

In [8]:
chunks = LazySeqDataset(hparams, path=f_path, keep_open=True)

In [10]:
chunks

In [12]:
len(chunks)

19010

In [14]:
chunks[0]

(0, tensor([1, 1, 9,  ..., 0, 1, 1]), tensor(15), 0)

In [16]:
chunks[0][1].shape

torch.Size([4096])

In [48]:
sample = chunks[2][1]
len(sample)

1180

As seen above, the chunks do not come padded -- the padding method below may not be the most efficient, but it'll get the job done for now -- may also want to go further up the data pipeline at some point

In [59]:
zs = torch.zeros(4096)
zs

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [71]:
#https://discuss.pytorch.org/t/how-to-do-padding-based-on-lengths/24442/2?u=aza
zs[:len(sample)] = sample

In [72]:
zs

tensor([10., 10., 10.,  ...,  0.,  0.,  0.])

In [24]:
from torch.utils.data import Dataset, DataLoader

In [64]:
class taxon_ds(Dataset):
    def __init__(self, chunks):
        self.chunks = chunks
    def __len__(self):
        return len(self.chunks)
    def __getitem__(self, idx):
        x = chunks[idx][1]
        if (len(x) < 4096):
            padded = torch.zeros(4096)
            padded[:len(x)] = x
            x = padded
        y = chunks[idx][2]
        return (x, y)

In [65]:
ds = taxon_ds(chunks)

In [66]:
ds[0]

(tensor([1, 1, 9,  ..., 0, 1, 1]), tensor(15))

That dataset above should be using a transform :D

In [75]:
dl = DataLoader(ds, batch_size=16, shuffle=True)
len(dl)

1189

In [76]:
batch = next(iter(dl))
len(batch), batch[0].shape, batch[1].shape

(2, torch.Size([16, 4096]), torch.Size([16]))

We now have functioning dataloaders!

In [78]:
sample = batch[0]

Let's see how we'll pass a batch through a convolutional layer -- will need to add a dimension to the tensor in order to provide the channel dimension that the conv layer is expecting

In [88]:
import torch.nn as nn
#torch.nn.Conv1d??

In [89]:
nn.Conv1d(1, 2, 3)(sample.unsqueeze(1)).shape

torch.Size([16, 2, 4094])

Looks like it's working :-P